In [1]:
import numpy as np
import pandas as pd

## 5.8.4

In [2]:
journal = pd.read_csv("./data/journal.csv")

In [3]:
n = 26
N = 1285
y_r = journal["nonprob"].sum() / journal["numemp"].sum()

v_y_r =(1/n - 1/N) * 1/(journal["numemp"].mean()**2) * np.sum((journal["nonprob"] - journal["numemp"] * y_r)**2) / (n-1)
se_y_r = np.sqrt(v_y_r)
print(y_r)
print(se_y_r)

0.9256756756756757
0.033986721952566884


## 5.8.5

In [4]:
spanish = pd.read_csv("./data/spanish.csv")

In [5]:
class OneStageCluster:
    def __init__(self, data, group, n, N):
        self.data = data
        self.data_group = data.groupby(group)
        self.n = n
        self.N = N
        self.Mi = self.data_group.count().iloc[:,0]
        
    def get_t_unb(self, y):
        return self.N / self.n * self.data[y].sum()
    
    def get_s_t_2(self, y):
        return self.data_group[y].sum().var(ddof=1)
    
    def get_var_t_unb(self, y):
        return self.N**2 * (1-self.n/self.N) * self.get_s_t_2(y) / self.n
    
    def get_se_t_unb(self, y):
        return np.sqrt(self.get_var_t_unb(y))
    
    def get_ci_t_unb(self, y):
        t_unb = self.get_t_unb(y)
        se = self.get_se_t_unb(y)
        return (t_unb - 1.96 * se, t_unb + 1.96 * se)
    
    def get_y_r(self, y):
        return np.sum(self.Mi * self.data_group[y].mean()) / np.sum(self.Mi)
    
    def get_s_r_2(self, y):
        return 1/(self.n-1) * np.sum(self.Mi**2 * (self.data_group[y].mean() - self.get_y_r(y))**2)
    
    def get_se_y_r(self, y):
        return np.sqrt((1-self.n/self.N) * self.get_s_r_2(y) / n / self.Mi.mean()**2)
    
    def get_ci_y_r(self, y):
        y_r = self.get_y_r(y)
        se = self.get_se_y_r(y)
        return (y_r - 1.96 * se, y_r + 1.96 * se)
    

In [6]:
Sp = OneStageCluster(spanish, "class", 10, 72)

In [7]:
print(Sp.get_t_unb("trip"))
print(Sp.get_ci_t_unb("trip"))

453.6
(234.4288772305987, 672.7711227694014)


In [8]:
print(Sp.get_y_r("score"))
print(Sp.get_ci_y_r("score"))

66.79591836734694
(63.5029015105676, 70.08893522412629)


## 5.8.6

In [9]:
cans = pd.DataFrame()
cans["Can1"] = [1,4,0,3,4,0,5,3,7,3,4,0]
cans["Can2"] = [5,2,1,6,9,7,5,0,3,1,7,0]
cans["Can3"] = [7,4,2,6,8,3,1,2,5,4,9,0]
n = 12
N = 580
m = 3
M = 24

In [10]:
class TwoStageCluster:
    def __init__(self, data, n, N, m, M):
        self.data = data
        self.n = n
        self.N = N
        self.m = m
        self.M = M
        self.w = self.N * self.M / (self.n * self.m) 

    def get_t_unb(self):
        return (self.w * self.data).sum().sum()
    
    def get_s_t_2(self):
        return 1/(self.n-1) * np.sum((self.data.sum(axis=1) - self.get_t_unb()/self.N)**2)
    
    def get_var_t_unb(self):
        s_i_2 = self.data.var(axis=1, ddof=1)
        s_t_2 = self.get_s_t_2()
        v = self.N**2 * (1-self.n/self.N) * s_t_2/self.n + self.N/self.n * (1-self.m/self.M) * self.M**2/self.m * np.sum(s_i_2)
        return v
    
    def get_se_t_unb(self):
        return np.sqrt(self.get_var_t_unb())
    
    def get_ci_t_unb(self):
        t = self.get_t_unb()
        se = self.get_se_t_unb()
        return (t - 1.96*se, t + 1.96*se)
    
    def get_var_approx(self):
        return self.N**2 * self.get_s_t_2() / n
    
    def get_anova_table(self):
        ssb = np.sum(self.m * (self.data.mean(axis=1) - self.get_t_unb() / (self.N * self.M))**2)
        ssw = np.sum(self.data.var(axis=1, ddof=self.m-1))
        ssto = ssw + ssb
        msb = ssb / (self.n-1)
        msw = ssw / (self.n * (self.m-1))
        msto = ssto / (self.n * self.m -1)
        d = pd.DataFrame()
        d["SS"] = [ssb, ssw, ssto]
        d["MS"] = [msb, msw, msto]
        d["df"] = [self.n-1, (self.n * (self.m-1)), (self.n * self.m -1)]
        return d
        
    def get_s_2_hat(self):
        ssb = np.sum(self.m * (self.data.mean(axis=1) - self.get_t_unb() / (self.N * self.M))**2)
        ssw = np.sum(self.data.var(axis=1, ddof=self.m-1))
        msb = ssb / (self.n-1)
        msw = ssw / (self.n * (self.m-1))
        s = self.M * (self.N-1) / (self.m * (self.N * self.M - 1)) * msb + ((self.m - 1) * self.N * self.M + self.M - self.m) / (self.m * (self.N * self.M)) * msw
        return s
    
    def get_icc(self):
        ssb = np.sum(self.m * (self.data.mean(axis=1) - self.get_t_unb() / (self.N * self.M))**2)
        ssw = np.sum(self.data.var(axis=1, ddof=self.m-1))
        msb = ssb / (self.n-1)
        msw = ssw / (self.n * (self.m-1))
        msb_hat = self.M / self.m * msb - (self.M/self.m-1) * msw
        icc = 1/(self.M-1) * (((self.N-1)*self.M*msb_hat)/((self.N * self.M -1)*self.get_s_2_hat())-1)
        return icc
    
    def get_r_a_2(self):
        ssw = np.sum(self.data.var(axis=1, ddof=self.m-1))
        msw = ssw / (self.n * (self.m-1))
        return 1-msw/(self.get_s_2_hat())
        
    def get_deff(self):
        ssb = np.sum(self.m * (self.data.mean(axis=1) - self.get_t_unb() / (self.N * self.M))**2)
        ssw = np.sum(self.data.var(axis=1, ddof=self.m-1))
        msb = ssb / (self.n-1)
        msw = ssw / (self.n * (self.m-1))
        msb_hat = self.M / self.m * msb - (self.M/self.m-1) * msw
        return msb_hat/self.get_s_2_hat()
    
    def get_msb_hat(self):
        ssb = np.sum(self.m * (self.data.mean(axis=1) - self.get_t_unb() / (self.N * self.M))**2)
        ssw = np.sum(self.data.var(axis=1, ddof=self.m-1))
        msb = ssb / (self.n-1)
        msw = ssw / (self.n * (self.m-1))
        msb_hat = self.M / self.m * msb - (self.M/self.m-1) * msw
        return msb_hat

In [11]:
Cans = TwoStageCluster(cans, 12, 580, 3, 24)

In [12]:
print(Cans.get_t_unb())
print(Cans.get_ci_t_unb())

50653.333333333336
(24617.811594461407, 76688.85507220526)


In [13]:
print(Cans.get_var_t_unb())
print(Cans.get_var_approx())

176449498.18181822
179726796.969697


## 5.8.7

In [14]:
m_i = np.array([10,4,7,8,2,3])
M_i = np.array([52,19,37,39,8,14])
city = np.array([1,2,3,4,5,6])
sell = np.array([146,180,251,152,72,181,171,361,73,186,
               99,101,52,121,
               199,179,98,63,126,87,62,
               226,129,57,46,86,43,85,165,
               12,23,
               87,43,59])
sold = pd.DataFrame()
sold["city"] = np.repeat(city, m_i)
sold["sell"] = sell

In [15]:
class UnequalTwoStageCluster:
    def __init__(self, data, group, y, n, N, mi, Mi):
        self.data = data
        self.data_group = data.groupby(group)
        self.n = n
        self.N = N
        self.mi = mi
        self.Mi = Mi
        self.y = y
        self.w_ij = self.N * self.Mi / (self.n * self.mi) 

    def get_t_unb(self):
        return (self.w_ij * self.data_group[self.y].sum()).sum()
    
    def get_s_t_2(self):
        return 1/(self.n-1) * np.sum((self.Mi * self.data_group[self.y].mean() - self.get_t_unb()/self.N)**2)
    
    def get_var_t_unb(self):
        s_i_2 = self.data_group[self.y].var(ddof=1)
        s_t_2 = self.get_s_t_2()
        v = self.N**2 * (1-self.n/self.N) * s_t_2/self.n + self.N/self.n * np.sum((1-self.mi/self.Mi) * self.Mi**2/self.mi * s_i_2)
        return v
    
    def get_se_t_unb(self):
        return np.sqrt(self.get_var_t_unb())
    
    def get_ci_t_unb(self):
        t = self.get_t_unb()
        se = self.get_se_t_unb()
        return (t - 1.96*se, t + 1.96*se)
    
    def get_y_r(self):
        return np.sum(self.Mi * self.data_group[self.y].mean()) / np.sum(self.Mi)
    
    def get_s_r_2(self):
        return 1/(self.n-1) * np.sum(self.Mi**2 * (self.data_group[self.y].mean() - self.get_y_r())**2)
    
    def get_var_y_r(self):
        s_i_2 = self.data_group[self.y].var(ddof=1)
        s_r_2 = self.get_s_r_2()
        v = 1/self.Mi.mean()**2 * (1-self.n/self.N) * s_r_2/self.n + 1/(self.n * self.N * self.Mi.mean()**2) * np.sum((1-self.mi/self.Mi) * self.Mi**2/self.mi * s_i_2)
        return v
    
    def get_se_y_r(self):
        return np.sqrt(self.get_var_y_r())


In [16]:
Sold = UnequalTwoStageCluster(sold, "city", "sell", 6, 45, m_i, M_i)

In [17]:
print(Sold.w_ij)

[39.         35.625      39.64285714 36.5625     30.         35.        ]


In [18]:
print(Sold.get_t_unb())
print(Sold.get_se_t_unb())

152972.2232142857
56781.080266243705


In [19]:
print(Sold.get_y_r())
print(Sold.get_se_y_r())

120.68814454775993
20.04892852479213


## 补充

In [20]:
print(Cans.get_anova_table())

           SS         MS  df
0  149.638889  13.603535  11
1  108.666667   4.527778  24
2  258.305556   7.380159  35


In [21]:
print(Cans.get_icc())
print(Cans.get_r_a_2())

0.40010757653283174
0.4001206706624777


In [22]:
print(Cans.get_deff())

10.219360911664591


In [23]:
print(Cans.get_s_2_hat())

7.547814295881868


In [24]:
Cans.get_msb_hat()

77.13383838383837